In [ ]:
#DORCs parameters
rnaRDS = "" # .h5 file 
atacFragFile = "" #.tsv fragment file
peakFile = "" #.bed peakset file
genome = "" #genome
nCores = 4
savePlotsToDir = TRUE
prefix = "prefix"

#RNA QC parameters
minFeature_RNA = 200 #Seurat QC for number of min features
maxFeature_RNA = 2500 #Seurat QC for number of max features
percentMT_RNA = 5 #Seurat QC for max % of mt 
minCells_RNA = 3 #Seurat QC for min number of cells

#ATAC QC parameter
fripCutOff = 0.3 #QC threshold for fRIP score
chunkSize = 50000 #chunk size (number of pairs) to parallelize centering ATAC counts 

#Background correlation parameters
numNearestNeighbor = 100 #Number of nearest neighbors
numBackgroundPairs = 1e+05 #Number of background gene-peak pairs to generate

#DORC genes parameters
windowPadSize = 50000 #Regulatory region around TSS. Default is +/- 50Kb
dorcGeneCutOff = 10 #No. sig peaks needed to be called a DORC
corrPVal = 0.05 #pval cutoff for correlation statistical test
topNGene = 20 #Label top N genes in j-Plot

In [ ]:
packages = c("dplyr","Seurat","patchwork","GenomicRanges","ggplot2","ggrepel","reshape2","ggrastr","BuenColors","foreach","iterators","parallel","Biostrings","logr")

if(genome == "hg38"){
    BiocManager::install("BSgenome.Hsapiens.UCSC.hg38", update=F, ask=F)
    packages = c(packages, "BSgenome.Hsapiens.UCSC.hg38")
} else if(genome == "mm10"){
    BiocManager::install("BSgenome.Mmusculus.UCSC.mm10", update=F, ask=F)
    packages = c(packages, "BSgenome.Mmusculus.UCSC.mm10")
}

new.packages <- packages[!(packages %in% installed.packages()[,"Package"])]
if(length(new.packages)) BiocManager::install(new.packages, update=F, ask=F)

suppressMessages(lapply(packages, library, character.only = TRUE))

options("logr.notes" = FALSE)

In [ ]:
#Source functions 

#download from gh and source?
source("/home/R/DORCS_helper_functions_optimized.R")
load("/home/R/TSSRanges.RData")

logfile <- file.path(paste0(prefix,".dorcs.logfile.",genome,".txt"))
lf <- log_open(logfile)

In [ ]:
#Create and preprocess RNA count matrix; using Seurat functions

rnaCounts = tryCatch({
        log_print("# Create RNA matrix")
    
        #Code start to create RNA matrix
    
        rnaCounts = Read10X_h5(rnaCountMatrix)
        rnaCounts = CreateSeuratObject(counts = rnaCounts, project = "shareseq", min.cells = minCells_RNA, min.features = minFeature_RNA)
        rnaCounts[["percent.mt"]] = PercentageFeatureSet(rnaCounts, pattern = "^MT-")
        RNAVlnPlot = VlnPlot(rnaCounts, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3)
        print(RNAVlnPlot)
    
        #rnaCounts = readRDS(rnaRDS)

        #QC filtering and normalization
        rnaCounts = subset(rnaCounts, subset = nFeature_RNA > minFeature_RNA & nFeature_RNA < maxFeature_RNA & percent.mt < percentMT_RNA)
        rnaCounts = NormalizeData(rnaCounts)
    
        #Code end to create RNA matrix
    
        log_print("SUCCESSFUL: Create RNA matrix")
        return(rnaCounts)
    },
    error = function(cond) {
        log_print("ERROR: Create RNA matrix")
        log_print(cond)
    }
)

In [ ]:
#Create scATACseq matrix

peaksSE = tryCatch({
        log_print("# Create scATACseq matrix")
    
        #Code start to create scATACseq matrix
    
        peaksSE = getCountsFromFrags(fragFile=atacFragFile, peakFile=peakFile)
        
        #Code end to create scATACseq matrix
    
        log_print("SUCCESSFUL: Create scATACseq matrix")
        return(peaksSE)
    },
    error = function(cond) {
        log_print("ERROR: Create scATACseq matrix")
        log_print(cond)
    }
)

In [ ]:
# Find common cells in RNA and ATAC; clean up before correlation step 

#filter by fRiP
SE.filt = tryCatch({
        log_print("# Filter by fRiP")
    
        #Code start to filter by fRiP
    
        SE.filt = peaksSE[,peaksSE$FRIP > fripCutOff]
        
        #Code end to filter by fRiP
    
        log_print("SUCCESSFUL: Filter by fRiP")
        return(SE.filt)
    },
    error = function(cond) {
        log_print("ERROR: Filter by fRiP")
        log_print(cond)
    }
)


#extract RNA count matrix
rnaMat = tryCatch({
        log_print("# Extract RNA count matrix")
    
        #Code start to extract RNA count matrix
    
        rnaMat <- rnaCounts[["RNA"]]@data
        
        #Code end to extract RNA count matrix
    
        log_print("SUCCESSFUL: Extract RNA count matrix")
        return(rnaMat)
    },
    error = function(cond) {
        log_print("ERROR: Extract RNA count matrix")
        log_print(cond)
    }
)


#clean up
tryCatch({
        log_print("# Clean up")
    
        #Code start to clean up
    
        rm(peaksSE)
        rm(rnaCounts)

        #Cleaning - change barcode names
        SE.filt$sample = sub(",P1\\.[0-9]+", "", SE.filt$sample)
        colnames(SE.filt) = sub(",P1\\.[0-9]+", "", colnames(SE.filt))
        colnames(SE.filt) = gsub(",", "\\.", colnames(SE.filt))
        colnames(rnaMat) = sub("\\,P1\\.[0-9]+", "", colnames(rnaMat))
        colnames(rnaMat) = gsub(",", "\\.", colnames(rnaMat))
        
        #Code end to clean up
    
        log_print("SUCCESSFUL: Clean up")
    },
    error = function(cond) {
        log_print("ERROR: Clean up")
        log_print(cond)
    }
)

#Get intersect of cells in RNA amd ATAC
cells = tryCatch({
        log_print("# Get intersect of cells in RNA amd ATAC")
    
        #Code start to get intersect of cells in RNA amd ATAC
    
        cells = intersect(colnames(SE.filt), colnames(rnaMat))
        
        #Code end to get intersect of cells in RNA amd ATAC
    
        log_print("SUCCESSFUL: Get intersect of cells in RNA amd ATAC")
        return(cells)
    },
    error = function(cond) {
        log_print("ERROR: Get intersect of cells in RNA amd ATAC")
        log_print(cond)
    }
)


In [ ]:
#Correlation

cisCor = tryCatch({
        log_print("# Correlation")
    
        #Code start to run correlation
    
        set.seed(123)

        #Run fast gene peak correlation
        cisCor <- fastGenePeakcorr(
          SE.filt[,cells],
          rnaMat[,cells],
          genome = genome, # This will be one of "hg19","hg38" or "mm10"
          windowPadSize = windowPadSize,
          normalizeATACmat = TRUE,
          nCores = nCores,
          p.cut = NULL,
          n_bg = numNearestNeighbor,
          n_BgPairs = numBackgroundPairs,
          chunkSize = chunkSize
        )
        
        #Code end to run correlation
    
        log_print("SUCCESSFUL: Correlation")
        return(cisCor)
    },
    error = function(cond) {
        log_print("ERROR: Correlation")
        log_print(cond)
    }
)

#filter by pval
cisCor.filt <- cisCor %>% dplyr::filter(pvalZ <= corrPVal)

In [ ]:
#dorcGenes and j-Plot

dorcGenes = tryCatch({
        log_print("# DORC genes and J-plot")
    
        #Code start to find DORC genes and create j-Plot
    
        dorcGenes = dorcJPlot(dorcTab = cisCor.filt,
                       cutoff = dorcGeneCutOff, # No. sig peaks needed to be called a DORC
                       labelTop = topNGene,
                       returnGeneList = TRUE, # Set this to FALSE for just the plot
                       force=2)
    
        #Code end to find DORC genes and create j-Plot
    
        log_print("SUCCESSFUL: DORC genes and J-plot")
        return(dorcGenes)
    },
    error = function(cond) {
        log_print("ERROR: DORC genes and J-plot")
        log_print(cond)
    }
)

In [ ]:
#Create final output files

tryCatch({
        log_print("# Final output files")
    
        #Code start to create final output files
    
        if(savePlotsToDir){
            dir.create("plots")
            savePlots = function(name, plotObject){
                filename = paste0(prefix,".dorcs.",name,".",genome)
                png(sprintf("plots/%s.png", filename), width=480*wf, height=480*hf)
                print(plotObject)
                dev.off()
            }
            savePlots("rna_violin_plot", RNAVlnPlot)
            savePlots("jplot", dorcJPlot(dorcTab = cisCor.filt,cutoff = dorcGeneCutOff, labelTop = topNGene,returnGeneList = FALSE,force=2))
        }

        write.table(dorcGenes, file=paste0(prefix,".dorcs.","dorc_genes_summary",".",genome,".csv"), row.names = T, quote = F, sep = ",")
        write.table(cisCor, file=paste0(prefix,".dorcs.","all_regions_summary",".",genome,".csv"), row.names = T, quote = F, sep = ",")

        files2zip <- dir('plots', full.names = TRUE)
        zip(zipfile = 'plots', files = files2zip)

        #Code end to create final output files
    
        log_print("SUCCESSFUL: Final output files")
    },
    error = function(cond) {
        log_print("ERROR: Final output files")
        log_print(cond)
    }
)

log_close()